In [1]:
%matplotlib inline
!pip install PyPortfolioOpt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
import seaborn as sns
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [2]:
stocks = pd.read_csv('adjprice.csv')
stocks['Date'] = pd.to_datetime(stocks['Date'], format='%Y%m%d')
StartDate = "1993/01/01"
EndDate = "1999/01/01"

In [3]:
#Stocks that last for the first 6 years
pd.options.mode.chained_assignment = None
stocks_short = stocks[(stocks["Date"] >= StartDate) & (stocks["Date"] <= EndDate)] #slicing based on chosen start and end dates.

#dropping any stocks that have N/A values as this means they didnt exist during the chosen time period
stocks_short.dropna(axis=1,inplace=True)

stocks_short

,Date,0111145D US Equity,0202445Q US Equity,0203524D US Equity,0226226D US Equity,0376152D US Equity,0544749D US Equity,0574018D US Equity,0598884D US Equity,0820143D US Equity,...,XEL US Equity,XL US Equity,XLNX US Equity,XOM US Equity,XRAY US Equity,XRX US Equity,XTO US Equity,YRCW US Equity,ZETHQ US Equity,ZION US Equity
0,1993-09-07,13.2719,13.6829,8.4429,8.1042,11.0000,17.8887,6.8315,28.1246,17.0742,...,6.9696,12.3149,2.5342,8.0198,5.3036,20.5503,1.0480,144439.5121,6.875,7.1675
1,1993-09-08,13.3263,13.5315,8.2147,7.9590,11.0000,17.8064,6.8315,27.5051,16.8567,...,6.9696,12.2506,2.4735,8.0815,5.1244,20.5503,1.0114,143691.1208,6.750,6.9766
2,1993-09-09,13.7070,13.3800,8.7852,8.0627,11.1250,17.6831,6.8315,27.7529,16.8567,...,6.9326,12.2506,2.5797,8.0044,5.1244,20.3040,0.9747,143691.1208,6.625,6.9554
3,1993-09-10,13.3807,13.4810,9.4127,8.0368,11.1250,17.6420,6.8773,27.5051,16.6392,...,7.0065,12.2506,2.7087,8.0969,5.3036,20.4096,0.9747,145187.9033,6.750,7.2099
4,1993-09-11,13.3807,13.4810,9.4127,8.0368,11.1250,17.6420,6.8773,27.5051,16.6392,...,7.0065,12.2506,2.7087,8.0969,5.3036,20.4096,0.9747,145187.9033,6.750,7.2099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,1998-12-28,23.1020,23.2066,6.0184,18.9359,5.4375,14.3580,11.4003,59.3125,14.9911,...,11.1435,41.3006,11.0281,22.2396,7.6541,109.7021,0.8498,118586.3002,0.270,44.1165
1939,1998-12-29,22.9618,23.7438,5.8473,19.4173,5.0625,14.5511,11.3633,59.3125,15.0530,...,11.1933,41.1520,10.7664,22.5754,7.8028,115.0564,0.8285,123038.3456,0.270,45.0890
1940,1998-12-30,22.9618,24.0124,6.0755,19.2033,5.3438,14.6235,11.5114,59.3125,14.9291,...,11.1610,42.5262,11.5289,22.2769,7.8028,113.7475,0.9006,123038.3456,0.270,46.5507
1941,1998-12-31,23.2367,24.3347,6.1611,19.2836,5.7500,14.7683,11.9925,59.3125,14.9911,...,11.2115,44.5689,11.8589,21.8291,7.6541,112.3198,1.0807,123847.8084,0.260,47.4541


In [4]:
#First 5 years of the dataframe above

StartDate = "1993/01/01"
EndDate = "1998/01/31"
stocks5 = stocks_short[(stocks_short["Date"] >= StartDate) & (stocks_short["Date"] <= EndDate)]
stocks5

,Date,0111145D US Equity,0202445Q US Equity,0203524D US Equity,0226226D US Equity,0376152D US Equity,0544749D US Equity,0574018D US Equity,0598884D US Equity,0820143D US Equity,...,XEL US Equity,XL US Equity,XLNX US Equity,XOM US Equity,XRAY US Equity,XRX US Equity,XTO US Equity,YRCW US Equity,ZETHQ US Equity,ZION US Equity
0,1993-09-07,13.2719,13.6829,8.4429,8.1042,11.0000,17.8887,6.8315,28.1246,17.0742,...,6.9696,12.3149,2.5342,8.0198,5.3036,20.5503,1.0480,144439.5121,6.8750,7.1675
1,1993-09-08,13.3263,13.5315,8.2147,7.9590,11.0000,17.8064,6.8315,27.5051,16.8567,...,6.9696,12.2506,2.4735,8.0815,5.1244,20.5503,1.0114,143691.1208,6.7500,6.9766
2,1993-09-09,13.7070,13.3800,8.7852,8.0627,11.1250,17.6831,6.8315,27.7529,16.8567,...,6.9326,12.2506,2.5797,8.0044,5.1244,20.3040,0.9747,143691.1208,6.6250,6.9554
3,1993-09-10,13.3807,13.4810,9.4127,8.0368,11.1250,17.6420,6.8773,27.5051,16.6392,...,7.0065,12.2506,2.7087,8.0969,5.3036,20.4096,0.9747,145187.9033,6.7500,7.2099
4,1993-09-11,13.3807,13.4810,9.4127,8.0368,11.1250,17.6420,6.8773,27.5051,16.6392,...,7.0065,12.2506,2.7087,8.0969,5.3036,20.4096,0.9747,145187.9033,6.7500,7.2099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,1998-01-27,21.6913,27.9457,11.8087,12.8370,9.5625,22.7327,13.1939,45.6661,20.9088,...,10.3999,34.9751,6.6009,17.5968,8.8084,74.3875,2.1325,174843.9647,6.6875,32.7415
1604,1998-01-28,21.8239,27.8401,12.3792,13.0102,9.7500,23.6401,13.2287,46.0410,20.9088,...,10.3038,35.3742,7.0107,17.2507,8.8453,75.6185,2.1680,178891.2787,7.0000,32.5063
1605,1998-01-29,21.7244,28.3155,12.3221,13.2632,9.7500,23.5238,13.1245,46.3533,21.5096,...,10.3158,35.5556,6.7944,17.7061,8.8821,76.0875,2.2509,173629.7705,6.8125,32.7886
1606,1998-01-30,21.3596,27.6816,12.8355,13.3697,8.7500,23.3144,13.0898,46.4158,21.3894,...,10.3038,35.4104,6.9082,17.2871,8.7715,75.3840,2.2273,169177.7251,6.6875,32.3652


In [5]:
# Read in price data
df = stocks5.set_index('Date')

# Calculate expected annual returns
er = expected_returns.mean_historical_return(df)

#sorts by expected return and then takes the top 20
er_sorted = pd.Series.sort_values(er,ascending=False)
port_stocks = er_sorted.head(10)
port_stocks

DELL US Equity        0.819461
COO US Equity         0.810868
MFE US Equity         0.602746
VTSS US Equity        0.564723
3029830Q US Equity    0.564576
TWX US Equity         0.548778
AN US Equity          0.490060
244630Q US Equity     0.479777
CDNS US Equity        0.458072
2968900Q US Equity    0.456480
dtype: float64

In [6]:
#Database of out portfolio with the top 10 expected return stocks for the past 5 years

# create a new dataframe with the first column being the dates
portfolio = pd.DataFrame(stocks5["Date"],columns=['Date'])
for col in stocks5.columns: # go through all the columns in our original data frame
    if col in port_stocks.index: # check if this column is in the names for our random sample
        portfolio = pd.concat([portfolio,stocks5[col]],axis=1) # if so, add the stock prices to the new dataframe
portfolio

,Date,244630Q US Equity,2968900Q US Equity,3029830Q US Equity,AN US Equity,CDNS US Equity,COO US Equity,DELL US Equity,MFE US Equity,TWX US Equity,VTSS US Equity
0,1993-09-07,2.1438,3.0826,2.0156,1.4990,2.5278,0.5491,0.2620,1.7778,0.5594,12.5000
1,1993-09-08,2.1438,3.0826,1.9219,1.4749,2.3889,0.5948,0.2676,1.6790,0.5713,12.5000
2,1993-09-09,2.1515,3.0826,1.8594,1.4749,2.4722,0.5948,0.2620,1.6790,0.6100,12.0833
3,1993-09-10,2.1899,3.0936,2.0313,1.4990,2.4722,0.6406,0.2639,1.6543,0.6189,11.6667
4,1993-09-11,2.1899,3.0936,2.0313,1.4990,2.4722,0.6406,0.2639,1.6543,0.6189,11.6667
...,...,...,...,...,...,...,...,...,...,...,...
1603,1998-01-27,23.6926,34.1561,34.8125,18.6737,25.0000,22.6957,11.6790,34.3333,8.6944,203.4375
1604,1998-01-28,24.7214,35.1469,36.3750,18.8429,28.5625,23.3364,11.6341,35.9583,9.0098,214.3750
1605,1998-01-29,26.3113,34.6791,34.8750,18.9558,27.6250,23.3058,11.8138,36.1667,9.0396,215.3125
1606,1998-01-30,26.0930,33.9084,35.0000,19.0686,28.0000,24.2210,11.9111,36.0000,9.1050,217.1875


# Make changes to the While loop with equal weights :P

In [21]:
pd.options.mode.chained_assignment = None
stop = False
start_year = 1993
end_year = 1999
count = 1
while stop == False:
    
    if end_year == 2019:
        stop = True
    
    stocks['Date'] = pd.to_datetime(stocks['Date'], format='%Y%m%d')
    
    StartDate = f"{start_year}/01/01"
    EndDate = f"{end_year}/01/01"

    #Stocks that last for the first 6 years

    stocks_short = stocks[(stocks["Date"] >= StartDate) & (stocks["Date"] <= EndDate)] #slicing based on chosen start and end dates.

    #dropping any stocks that have N/A values as this means they didnt exist during the chosen time period
    stocks_short.dropna(axis=1,inplace=True)

    #First 5 years of the dataframe above

    StartDate = f"{start_year}/01/01"
    EndDate = f"{end_year - 1}/01/31"
    stocks5 = stocks_short[(stocks_short["Date"] >= StartDate) & (stocks_short["Date"] <= EndDate)]

    # Read in price data
    df = stocks5.set_index('Date')

    # Calculate expected annual returns
    er = expected_returns.mean_historical_return(df)

    #sorts by expected return and then takes the top 20
    er_sorted = pd.Series.sort_values(er,ascending=False)
    port_stocks = er_sorted.head(10)

    #Database of out portfolio with the top 10 expected return stocks for the past 5 years

    # create a new dataframe with the first column being the dates
    portfolio = pd.DataFrame(stocks5["Date"],columns=['Date'])
    for col in stocks5.columns: # go through all the columns in our original data frame
        if col in port_stocks.index: # check if this column is in the names for our random sample
            portfolio = pd.concat([portfolio,stocks5[col]],axis=1) # if so, add the stock prices to the new dataframe



    # Read in price data
    df = portfolio.set_index('Date')

    # Calculate expected returns and sample covariance
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    # Calculate performance for equal weights

    ef = EfficientFrontier(mu, S)

    # Equal weights
    values = ((1/len(df.columns)) for _ in range(len(df.columns)))
    my_dict = {}

    for key, value in zip(df.columns,values):
        my_dict[key] = value

    my_dict

    ef.set_weights(my_dict)
    cleaned_weights = ef.clean_weights()
    ef.set_weights(cleaned_weights)
    weights5 = ef.weights
    values = ef.portfolio_performance()
    port_exp = values[0]

    #The following year

    StartDate = f"{end_year - 1}/01/01"
    EndDate = f"{end_year}/01/01"
    
    stocks_next = stocks_short[(stocks_short["Date"] >= StartDate) & (stocks_short["Date"] <= EndDate)]

    portfolio_next = pd.DataFrame(stocks_next["Date"],columns=['Date'])
    for col in stocks_next.columns: # go through all the columns in our original data frame
        if col in port_stocks.index: # check if this column is in the names for our random sample
            portfolio_next = pd.concat([portfolio_next,stocks_next[col]],axis=1) # if so, add the stock prices to the new dataframe


    df = portfolio_next.set_index('Date')

    # Calculate expected returns and sample covariance
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)

    print(f"Portfolio Year {end_year-1} - {end_year}:")
    print(f"Expected Return from Portfolio: {port_exp}")
    print(f"Actual Return for following Year: {mu.dot(weights5)}")
    print("")
    
    start_year += 1
    end_year += 1

Portfolio Year 1998 - 1999:
Expected Return from Portfolio: 0.5795540855321554
Actual Return for following Year: 0.501698615634609

Portfolio Year 1999 - 2000:
Expected Return from Portfolio: 0.6276270866023331
Actual Return for following Year: 1.1591310308654745

Portfolio Year 2000 - 2001:
Expected Return from Portfolio: 0.8329055789165631
Actual Return for following Year: -0.11080285337902823

Portfolio Year 2001 - 2002:
Expected Return from Portfolio: 0.6114327503647592
Actual Return for following Year: -0.3248862790464586

Portfolio Year 2002 - 2003:
Expected Return from Portfolio: 0.4275560655346838
Actual Return for following Year: -0.2773899730655856

Portfolio Year 2003 - 2004:
Expected Return from Portfolio: 0.3064170789856076
Actual Return for following Year: 0.2988195062042797

Portfolio Year 2004 - 2005:
Expected Return from Portfolio: 0.3625406431612244
Actual Return for following Year: 0.24641113252787628

Portfolio Year 2005 - 2006:
Expected Return from Portfolio: 0.375